In [ ]:
from brian2 import *
%matplotlib widget
prefs.codegen.target = 'numpy'

In [ ]:
start_scope()
tau_m_e = 20*ms; V_t_e = -55*mV
tau_m_i = 5*ms; V_t_i = -50*mV
tau_syn = 3*ms
E_L = -70*mV
w_ee = 0.001*mV
w_ie = 0.002*mV
w_ei = 0.0015*mV
w_ii = 0.001*mV
N_E, N_I = 800, 200

exc_eqs = '''
dV/dt = ((V - E_L) + I_syn) / tau_m : volt (unless refractory)
tau_m : second
dI_syn/dt = -I_syn/tau_syn : volt'''

inh_eqs = '''
dV/dt = ((V - E_L) + I_syn) / tau_m : volt (unless refractory)
tau_m : second
dI_syn/dt = -I_syn/tau_syn : volt'''

exc_neurons = NeuronGroup(N_E, exc_eqs,
                          threshold='V > V_t_e', reset='V = E_L', refractory=5*ms,
                          method='exact')
exc_neurons.V = 'E_L + rand()*30*mV'
exc_neurons.tau_m = 'tau_m_e - 2*ms + rand()*4*ms'

inh_neurons = NeuronGroup(N_I, inh_eqs,
                          threshold='V > V_t_i', reset='V = E_L', refractory=5*ms,
                          method='exact')
inh_neurons.V = 'E_L + rand()*30*mV'
inh_neurons.tau_m = 'tau_m_i - 1*ms + rand()*2*ms'

syn_ee = Synapses(exc_neurons, exc_neurons, '', on_pre='I_syn_post += w_ee')
syn_ee.connect(p=0.02)
syn_ei = Synapses(exc_neurons, inh_neurons, '', on_pre= 'I_syn_post += w_ei')
syn_ei.connect(p=0.01)
syn_ii = Synapses(inh_neurons, inh_neurons, '', on_pre= 'I_syn_post += w_ii')
syn_ii.connect(p=0.01)
syn_ie = Synapses(inh_neurons, exc_neurons, '', on_pre= 'I_syn_post += w_ie')
syn_ie.connect(p=0.04)

exc_spikes = SpikeMonitor(exc_neurons)
inh_spikes = SpikeMonitor(inh_neurons)

run(1*second, report='text')

In [ ]:
fig, (ax_top, ax_bottom) = plt.subplots(2, 1, sharex=True, gridspec_kw={'height_ratios': [N_E, N_I]})
ax_top.plot(exc_spikes.t/ms, exc_spikes.i, '|')
ax_bottom.plot(inh_spikes.t/ms, inh_spikes.i, '|')
ax_bottom.set(xlabel='time (ms)');